# Intalling necessary webscraping tools

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge html5lib --yes
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forg

In [4]:
#importing libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import lxml
import html5lib
import requests
print("Libraries imported.")

Libraries imported.


Use beautifulsoup to get the content of page and then find the table inside the content. After that, put it into a dataframe

In [5]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#Getting content of the page
soup = BeautifulSoup(res.content,'lxml')

#Finding table in the content
table = soup.find_all('table')[0] 

#Put the table into pandas dataframe
df = pd.read_html(str(table))[0]
df.rename(columns={'Postcode':'PostalCode'}, inplace=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [6]:
# Dropping the rows that has 'not assigned' value in both Borough and Neighborhood cells
df_drop = df.drop(df[(df['Borough'] =='Not assigned') & (df['Neighborhood'] == 'Not assigned')].index)

#Reset indexes
df_drop.reset_index(drop=True, inplace=True)

# if a row has a Borough and Neighborhood is 'Not asssigned', change neighborhood to Borough's name.
for i in range(df_drop.shape[0]):
    if df_drop.at[i, 'Neighborhood'] == 'Not assigned':
        df_drop.at[i, 'Neighborhood']=df_drop.at[i, 'Borough']

df_drop.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [7]:
#Neighborhoods that have same postal codes go into the same cell.
df_grouped = df_drop.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df_grouped

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [8]:
df_grouped.shape

(103, 3)

# In this part, dataframe will be improved by adding coordinates

In [14]:
#Using geospatial csv file that is provided for coordinates
!wget -q -O 'coordinates.csv' http://cocl.us/Geospatial_data

geo_cord = pd.read_csv('coordinates.csv')
geo_cord.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_cord

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [17]:
#Merging two dataframes

merged_df = pd.merge(df_grouped, geo_cord, on='PostalCode')
merged_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
